# Question 2: Resampling and Frequency Conversion

This question focuses on resampling operations and frequency conversion using ICU monitoring data (hourly) and patient vital signs data (daily).

## Setup

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import os

# Set random seed for reproducibility
np.random.seed(42)

# Set plotting style
plt.style.use('default')
sns.set_style('whitegrid')

# Create output directory
os.makedirs('output', exist_ok=True)

## Part 2.1: Load and Prepare Data

**Note:** These datasets have realistic characteristics:
- **ICU Monitoring**: 75 patients with variable stay lengths (2-30 days). Not all patients are present for the entire 6-month period - patients are admitted and discharged at different times.
- **Patient Vitals**: Already contains some missing visits (~5% missing data). This is realistic and will be useful for practicing missing data handling.

In [8]:
# Load ICU monitoring data (hourly)
icu_monitoring = pd.read_csv('data/icu_monitoring.csv')

# Load patient vitals data (daily) - for comparison
patient_vitals = pd.read_csv('data/patient_vitals.csv')

print("ICU monitoring shape:", icu_monitoring.shape)
print("Patient vitals shape:", patient_vitals.shape)

# Convert datetime columns and set as index
icu_monitoring['datetime'] = pd.to_datetime(icu_monitoring['datetime'])
icu_monitoring = icu_monitoring.set_index('datetime')

patient_vitals['date'] = pd.to_datetime(patient_vitals['date'])
patient_vitals = patient_vitals.set_index('date')

print("\nICU monitoring sample:")
print(icu_monitoring.head())
print("\nPatient vitals sample:")
print(patient_vitals.head())

# Check data characteristics
print(f"\nICU patients: {icu_monitoring['patient_id'].nunique()}")
print(f"ICU date range: {icu_monitoring.index.min()} to {icu_monitoring.index.max()}")
print(f"\nPatient vitals patients: {patient_vitals['patient_id'].nunique()}")
print(f"Patient vitals date range: {patient_vitals.index.min()} to {patient_vitals.index.max()}")

ICU monitoring shape: (27027, 7)
Patient vitals shape: (63624, 7)

ICU monitoring sample:
                    patient_id  heart_rate  blood_pressure_systolic  \
datetime                                                              
2023-03-26 00:00:00     ICU001   75.487292               119.658194   
2023-03-26 01:00:00     ICU001   72.404404               109.222567   
2023-03-26 02:00:00     ICU001   83.172760               109.448507   
2023-03-26 03:00:00     ICU001   86.241612               119.389839   
2023-03-26 04:00:00     ICU001   74.536290               119.072058   

                     blood_pressure_diastolic  oxygen_saturation  temperature  
datetime                                                                       
2023-03-26 00:00:00                 83.260736         100.000000         98.1  
2023-03-26 01:00:00                 75.955797          96.944358         98.3  
2023-03-26 02:00:00                 76.113955          98.137873         98.0  
2023-03-26 0

## Part 2.2: Time Series Selection

**⚠️ WARNING: Sort Index Before Date Selection!**
Since multiple patients share the same date, the `patient_vitals` index is non-monotonic (not strictly increasing). **You MUST sort the index first** before using `.loc` with date ranges:

In [9]:
patient_vitals = patient_vitals.sort_index()

Without sorting, pandas cannot reliably handle date range selections and may return unexpected results or errors.

**TODO: Perform time series indexing and selection**

In [10]:
# TODO: Select data by specific dates
# Note: Not all patients may have data on January 1, 2023 (some start later)
# Important: Sort the index first since multiple patients share the same date
patient_vitals = patient_vitals.sort_index()  # Sort for reliable date-based selection
# Select January 1, 2023 from patient_vitals
january_first = patient_vitals.loc['2023-01-01']
print("January 1, 2023 data:", january_first)
print(f"Records on Jan 1: {len(january_first)} (some patients may start later)")

# TODO: Select data by date ranges
# Select entire January 2023
january_data = patient_vitals.loc['2023-01-01':'2023-01-31']  
print("January 2023 shape:", january_data.shape)

# TODO: Select data by time periods
first_quarter = patient_vitals.loc['2023-01-01':'2023-03-31']  # Select Q1 2023
entire_year = patient_vitals.loc['2023']  # Select all of 2023 (will include patients with partial year data)

# TODO: Select first and last periods using .loc
first_week = patient_vitals.loc[:patient_vitals.index.min() + pd.Timedelta(days=6)]  # First 7 days
last_week = patient_vitals.loc[patient_vitals.index.max() - pd.Timedelta(days=6):]  # Last 7 days

# TODO: Use truncate() method
# Note: truncate() requires a sorted index. Sort first if needed: patient_vitals = patient_vitals.sort_index()
data_after_june = patient_vitals.truncate(before='2023-06-01')  # Truncate before June 1, 2023
data_before_september = patient_vitals.truncate(after='2023-08-31')   # Truncate after August 31, 2023

# TODO: Use selected data for analysis
# Compare average temperature between first quarter and data after June
print(f"\nFirst quarter average temperature: {first_quarter['temperature'].mean():.2f}°F")
print(f"After June average temperature: {data_after_june['temperature'].mean():.2f}°F")
print(f"First week average temperature: {first_week['temperature'].mean():.2f}°F")
print(f"Last week average temperature: {last_week['temperature'].mean():.2f}°F")

# For ICU data with time components:
# TODO: Select business hours (9 AM to 5 PM)
business_hours = icu_monitoring.between_time('09:00', '17:00')  # Use between_time()
print("Business hours data shape:", business_hours.shape)

# TODO: Select specific time (noon readings)
noon_data = icu_monitoring.at_time('12:00')  # Use at_time('12:00')

# TODO: Use time-based selection for analysis
# Compare vital signs during business hours vs other times
all_hours_avg = icu_monitoring.select_dtypes(include=[np.number]).mean()
business_hours_avg = business_hours.select_dtypes(include=[np.number]).mean()
print(f"\nAverage heart rate - All hours: {all_hours_avg['heart_rate']:.1f} bpm")
print(f"Average heart rate - Business hours: {business_hours_avg['heart_rate']:.1f} bpm")
print(f"Average temperature - All hours: {all_hours_avg['temperature']:.1f}°F")
print(f"Average temperature - Business hours: {business_hours_avg['temperature']:.1f}°F")

January 1, 2023 data: patient_id                     P0060
temperature                     98.6
heart_rate                  82.47421
blood_pressure_systolic          111
blood_pressure_diastolic          76
weight                          72.7
Name: 2023-01-01 00:00:00, dtype: object
Records on Jan 1: 6 (some patients may start later)
January 2023 shape: (1446, 6)

First quarter average temperature: 98.87°F
After June average temperature: 98.23°F
First week average temperature: 98.62°F
Last week average temperature: 98.46°F
Business hours data shape: (10107, 6)

Average heart rate - All hours: 81.2 bpm
Average heart rate - Business hours: 80.0 bpm
Average temperature - All hours: 99.0°F
Average temperature - Business hours: 99.0°F


## Part 2.3: Resampling Operations

**TODO: Perform resampling and frequency conversion**

**Important Note:** When resampling DataFrames that contain non-numeric columns (like `patient_id`), you'll get an error if you try to aggregate them with numeric functions like `mean()`. Use `df.select_dtypes(include=[np.number])` to select only numeric columns before resampling, or specify which columns to aggregate in `.agg()`.

In [11]:
# TODO: Resample hourly ICU data to daily
# Note: Exclude non-numeric columns like 'patient_id' when resampling
# Select only numeric columns before resampling
numeric_cols = icu_monitoring.select_dtypes(include=[np.number]).columns
icu_daily = icu_monitoring[numeric_cols].resample('D').mean()
print("ICU daily shape:", icu_daily.shape)

# TODO: Resample daily patient data to weekly
# Note: Exclude 'patient_id' column when resampling
# Select only numeric columns before resampling
numeric_cols_pv = patient_vitals.select_dtypes(include=[np.number]).columns
patient_vitals_weekly = patient_vitals[numeric_cols_pv].resample('W').mean()
print("Weekly resampled shape:", patient_vitals_weekly.shape)

# TODO: Resample daily patient data to monthly
patient_vitals_monthly = patient_vitals[numeric_cols_pv].resample('M').mean() # Resample to monthly with mean aggregation (use freq='ME' for Month End)
print("Monthly resampled shape:", patient_vitals_monthly.shape)

# TODO: Use different aggregation functions (mean, sum, max, min)
icu_daily_stats = icu_monitoring[numeric_cols].resample('D').agg(['mean', 'sum', 'max', 'min'])
print("ICU daily stats shape:", icu_daily_stats.shape)  
# Resample with multiple aggregations
# Example: resample('D').agg({'heart_rate': ['mean', 'max', 'min'], 
#                             'temperature': 'mean'})

# TODO: Handle missing values during resampling
# Demonstrate upsampling (monthly to daily) creates missing values
# Note: When upsampling, use .asfreq() to create missing values, or use .resample() with aggregation
monthly_to_daily = patient_vitals_monthly.resample('D').asfreq() # Upsample monthly data to daily (use .asfreq() or .resample('D'))
print("Missing values after upsampling from monthly to daily:")
print("Missing values after upsampling:", monthly_to_daily.isna().sum())

# TODO: Compare different resampling frequencies
# Create a DataFrame comparing resampling results at different frequencies
# Important: Since patient_vitals contains multiple patients per date, you need to aggregate by date first
# to create a single daily time series for comparison.
# Why aggregation is needed: The patient_vitals DataFrame has multiple rows per date (one for each patient),
# so we need to average across patients for each date to create a single daily time series that can be
# meaningfully compared with the weekly and monthly resampled data. Without aggregation, resampling would
# operate on each patient's time series separately, making it difficult to compare frequencies meaningfully.
# Steps:
# 1. Since 'date' is currently the index, reset it to a column first, then aggregate by date
#    Note: groupby('date').mean() automatically sets 'date' as the index in the result, so you don't need
#    to call set_index('date') again after groupby.
patient_vitals_reset = patient_vitals[numeric_cols_pv].reset_index()
patient_vitals_daily_agg = patient_vitals_reset.groupby('date').mean()
print("Aggregated daily patient vitals shape:", patient_vitals_daily_agg.shape)
#    # The date is already the index after groupby, so no need to set_index again
# 2. Compare the aggregated daily data with weekly and monthly resampled data
# Use patient_vitals data resampled to different frequencies:
# - Original daily data (aggregated by date): patient_vitals_daily_agg
# - Weekly resampled (patient_vitals_weekly) 
# - Monthly resampled (patient_vitals_monthly)
# Include columns: frequency, date_range, row_count, mean_temperature, std_temperature
# Use the 'temperature' column from each resampled dataset
# Example structure:
resampling_comparison = pd.DataFrame({
    'frequency': ['daily', 'weekly', 'monthly'],
    'date_range': [
        f"{patient_vitals_daily_agg.index.min().date()} to {patient_vitals_daily_agg.index.max().date()}",
        f"{patient_vitals_weekly.index.min().date()} to {patient_vitals_weekly.index.max().date()}",
        f"{patient_vitals_monthly.index.min().date()} to {patient_vitals_monthly.index.max().date()}"
    ],
    'row_count': [
        len(patient_vitals_daily_agg),
        len(patient_vitals_weekly),
        len(patient_vitals_monthly)
    ],
    'mean_temperature': [
        patient_vitals_daily_agg['temperature'].mean(),
        patient_vitals_weekly['temperature'].mean(),
        patient_vitals_monthly['temperature'].mean()
    ],
    'std_temperature': [
        patient_vitals_daily_agg['temperature'].std(),
        patient_vitals_weekly['temperature'].std(),
        patient_vitals_monthly['temperature'].std()
    ]
})

print("\nResampling comparison:")
print(resampling_comparison)

# TODO: Save results as 'output/q2_resampling_analysis.csv'
resampling_comparison.to_csv('output/q2_resampling_analysis.csv', index=False)
print("\nResampling analysis saved to 'output/q2_resampling_analysis.csv'")

ICU daily shape: (143, 5)
Weekly resampled shape: (53, 5)
Monthly resampled shape: (12, 5)
ICU daily stats shape: (143, 20)
Missing values after upsampling from monthly to daily:
Missing values after upsampling: temperature                 323
heart_rate                  323
blood_pressure_systolic     323
blood_pressure_diastolic    323
weight                      323
dtype: int64
Aggregated daily patient vitals shape: (365, 5)

Resampling comparison:
  frequency                date_range  row_count  mean_temperature  \
0     daily  2023-01-01 to 2023-12-31        365         98.481643   
1    weekly  2023-01-01 to 2023-12-31         53         98.484639   
2   monthly  2023-01-31 to 2023-12-31         12         98.486513   

   std_temperature  
0         0.354911  
1         0.352725  
2         0.364940  

Resampling analysis saved to 'output/q2_resampling_analysis.csv'


/var/folders/6w/gv8g6zzd417_9g9fl64d_nhc0000gn/T/ipykernel_66766/1828878904.py:16: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  patient_vitals_monthly = patient_vitals[numeric_cols_pv].resample('M').mean() # Resample to monthly with mean aggregation (use freq='ME' for Month End)


## Part 2.4: Missing Data Handling

**💡 TIP: High Percentage of Missing Data is Expected!**
When upsampling from monthly to daily frequency, you'll create approximately 96% missing data (only 12 month-end dates have values out of 365 days). This is normal and expected for upsampling - don't be alarmed!

**Approach:** Create missing values by upsampling monthly data to daily frequency. This creates a clear, structured pattern of missing data that's ideal for practicing imputation methods.

**TODO: Handle missing data in time series**

In [12]:
# TODO: Identify missing values in time series
# Use the monthly resampled data from Part 2.3 and upsample to daily:
#   - Take patient_vitals_monthly['temperature']
#   - Upsample to daily frequency using .resample('D').asfreq()
#   - This creates missing values for all days except month-end dates (~96% missing)
ts_with_missing = patient_vitals_monthly['temperature'].resample('D').asfreq()  # Time series with missing values  
print("Missing value count:", ts_with_missing.isna().sum())
print("Missing value percentage:", ts_with_missing.isna().sum() / len(ts_with_missing) * 100)

# TODO: Use forward fill and backward fill
ts_ffill = ts_with_missing.ffill()  # Forward fill missing values (use .ffill() method)
ts_bfill = ts_with_missing.bfill()  # Backward fill missing values (use .bfill() method)

# TODO: Use interpolation methods
ts_interpolated = ts_with_missing.interpolate() # Interpolate missing values
ts_interpolated_linear = ts_with_missing.interpolate(method='linear')  # Linear interpolation
ts_interpolated_time = ts_with_missing.interpolate(method='time')  # Time-based interpolation

# TODO: Use rolling mean for imputation
ts_rolling_imputed = ts_with_missing.rolling(window=3, min_periods=1).mean()  # Fill missing with rolling mean

# TODO: Create missing data report
# Document your missing data handling with the following sections:
# 1. Missing value summary: Total count and percentage
# 2. Missing data patterns: When/why data is missing (by month, day of week, etc.)
# 3. Imputation method: Which method you used (forward fill, backward fill, interpolation, rolling mean)
# 4. Rationale: Why you chose that method
# 5. Pros and cons: Advantages and limitations of your approach
# 6. Example: Show at least one example of missing data before and after imputation
# Minimum length: 300 words

# TODO: Document missing data patterns
# Analyze when/why data is missing
missing_by_month = ts_with_missing.groupby(ts_with_missing.index.month).apply(lambda x: x.isna().sum())
missing_by_day = ts_with_missing.groupby(ts_with_missing.index.dayofweek).apply(lambda x: x.isna().sum())
missing_patterns = f"Missing by month:\n{missing_by_month}\n\nMissing by day of week:\n{missing_by_day}"

missing_data_report = f"""

1) Missing value summary:
Total missing values: {ts_with_missing.isna().sum()}
Percentage missing: {ts_with_missing.isna().sum() / len(ts_with_missing) * 100:.2f}%

- There are 323 missing values and the missing value percentage is 96.4%.

2) Imputation methods applied for handling missing data:
- Forward fill (ts_ffill)
- Backward fill (ts_bfill)
- Linear interpolation (ts_interpolated_linear)
- Time-based interpolation (ts_interpolated_time)
- Rolling mean imputation with 3-day window (ts_rolling_imputed)

3) Why did you choose that method?
- Forward/backward fill maintains last known value and is simple for short gaps.
- Interpolation estimates missing values based on surrounding known points.
- Rolling mean smooths out fluctuations and reduces noise in imputed values.

4) What are the pros/cons of your approach?
- Forward/backward fill: 
    Pros: Simple and fast approach
    Cons: Can lead to biased distribution and errors if there is ectensive missingness. Not a suitable method for long gaps or trends. Backward fill cannot fill the last missing values.
- Interpolation: 
    Pros: Captures trends and better maintains data distribution and relationships between variables
    Cons: More complex, may not work well for non-linear trends or high volatility
- Rolling mean: 
    Pros: Reduces noise and smooths imputed values, suitable for time-series data
    Cons: More complex, may lag behind actual values, choice of window size affects results, trends to underestimate the true variance

5) Examples:
Original missing data (first 10 days of January):
{ts_with_missing.head(10)}

Forward fill imputed values:
{ts_ffill.head(10)}

Linear interpolation imputed values:
{ts_interpolated_linear.head(10)}
"""

# TODO: Save results as 'output/q2_missing_data_report.txt'
with open('output/q2_missing_data_report.txt', 'w') as f:
    f.write(missing_data_report)
    f.write(f"\n\nMissing patterns:\n{missing_patterns}")

Missing value count: 323
Missing value percentage: 96.41791044776119


## Submission Checklist

Before moving to Question 3, verify you've created:

- [ ] `output/q2_resampling_analysis.csv` - resampling analysis results
- [ ] `output/q2_missing_data_report.txt` - missing data handling report
